In [3]:
import pandas as pd
import math

In [ ]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code'] # Các cột thông tin của user
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp'] # Các cột thông tin rating của phim
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'] #Các item của phim

In [ ]:
# Đọc các file liên quan đến user, rating, phim
users = pd.read_csv("./ml-100k/u.user", sep="|", names= u_cols)
rating_base = pd.read_csv("./ml-100k/ua.base",sep="\t",names = r_cols)
rating_test = pd.read_csv("./ml-100k/ua.test",sep="\t",names = r_cols)
items = pd.read_csv("./ml-100k/u.item", sep="|",names=i_cols,encoding="latin-1")

# Chuyển thành dạng ma trận
rate_base = rating_base.to_numpy()
rate_test = rating_test.to_numpy()
X0 = items.to_numpy()
n_users = users.shape[0]
n_items = items.shape[0]

In [25]:
print("Một vài thông tin rating:")
print(rating_base.head())
print("\n\nMa trận rate_base:")
print(rate_base)
print("\n\nMa trận rate_test")
print(rate_test)

print(f"Chiều của rate_base: {rate_base.shape[0]}")
print(f"Chiều của rate_test: {rate_test.shape[0]}")
print()
print(f"Số lượng items: {n_items}")

Một vài thông tin rating:
   user_id  movie_id  rating  unix_timestamp
0        1         1       5       874965758
1        1         2       3       876893171
2        1         3       4       878542960
3        1         4       3       876893119
4        1         5       3       889751712


Ma trận rate_base:
[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


Ma trận rate_test
[[        1        20         4 887431883]
 [        1        33         4 878542699]
 [        1        61         4 878542420]
 ...
 [      943       570         1 888640125]
 [      943       808         4 888639868]
 [      943      1067         2 875501756]]
Chiều của rate_base: 90570
Chiều của rate_test: 9430

Số lượng items: 1682


In [ ]:
print(X0) # Đầy đủ thông tin như tên, ngày xuất bản, thể loại

[[1 'Toy Story (1995)' '01-Jan-1995' ... 0 0 0]
 [2 'GoldenEye (1995)' '01-Jan-1995' ... 1 0 0]
 [3 'Four Rooms (1995)' '01-Jan-1995' ... 1 0 0]
 ...
 [1680 'Sliding Doors (1998)' '01-Jan-1998' ... 0 0 0]
 [1681 'You So Crazy (1994)' '01-Jan-1994' ... 0 0 0]
 [1682 'Scream of Stone (Schrei aus Stein) (1991)' '08-Mar-1996' ... 0 0
  0]]


In [ ]:
print(X0[:,5:]) # 
X_train_counts = X0[:,5:] # Chỉ lấy giá trị là thể loại

[[0 0 0 ... 0 0 0]
 [0 1 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True,norm='l2') # l1 la MAE, l2 la MSE
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()
print(tfidf) # Mỗi hàng sẽ ứng với feature vector của 1 bộ phim

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.54 0.65 ... 0.54 0.   0.  ]
 [0.   0.   0.   ... 1.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [33]:
import numpy as np
y = rate_base[:,0]
print("Lấy ra id của tất cả user")
print(y)
print(np.where(y==2)) # Dùng np.where() sẽ trả về 1 mảng array chứa vị trí có giá trị tương ứng cần tìm
# Ví dụ với y == 2 thì có các vị trí như 262 -> 313 là chứa y == 2
print(np.where(y==2)[0]) # Phải thêm [0] để có thể lấy ra list 


Lấy ra id của tất cả user
[  1   1   1 ... 943 943 943]
(array([262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274,
       275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287,
       288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300,
       301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313]),)
[262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279
 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297
 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313]


In [ ]:
def get_items_rated_by_user(rate_matrix, user_id): # Hàm trả về item và score được đánh giá bởi người dùng user_id
    """
    Ở mỗi hàng của rate_matrix, ta có các thông tin như: user_id, item_id, rating (scores), time_stamp
    ta chỉ cần quan tâm đến 3 thông tin đầu
    trả về (item_ids, scores) đã được đánh giá bởi người có id là user_id
    rate_matrix ở đây là ma trận rate_base trước đó!
    """
    y = rate_matrix[:,0] # lấy tất cả user_id
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # while index in python starts from 0
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0 
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

In [45]:
print(tfidf.shape)


(1682, 19)


In [ ]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] # Chiều của dữ liệu
W = np.zeros((d, n_users)) # Ma trận 0 với size d x n_users
b = np.zeros((1, n_users)) # Ma trận 0 với size 1 x n_users

for i in range(n_users):    
    ids, scores = get_items_rated_by_user(rate_base, i) # Lấy ra được id và score mà người thứ i đã đánh giá
    clf = Ridge(alpha=0.01, fit_intercept  = True) # Khai báo mô hình hồi quy Ridge 
    Xhat = tfidf[ids, :] # Lấy ra các dòng tương ứng phim người thứ i đã xem
    clf.fit(Xhat, scores) # Huấn luyện dựa trên đặc trưng phim và điểm số
    W[:, i] = clf.coef_  # Trọng số 
    b[0, i] = clf.intercept_ # bias
    
    #print(f"Người thứ {i}:\nĐã đánh giá các phim có id:{ids}\n Và scores lần lượt: {scores}\n")

In [10]:
Yhat = tfidf.dot(W) + b
n = 10
np.set_printoptions(precision=2) # 2 digits after . 
ids, scores = get_items_rated_by_user(rate_test, n)
Yhat[n, ids]
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [ 37 109 110 226 424 557 722 724 731 739]
True ratings     : [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]


In [11]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return math.sqrt(se/cnt)

print ('RMSE for training:', evaluate(Yhat, rate_base, W, b))
print ('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.9089804562826721
RMSE for test    : 1.2703282700393035
